Nearest Neighbor 

In [8]:
import random
import itertools
import os

In [31]:
class NearestNeighbor:
    def __init__(self, num_cities):
        self.num_cities = num_cities 
        self.adj_list = {city: {} for city in range(self.num_cities)}
        self.generate_random_graph()
        
    def add_distance(self, city1, city2, distance): 
        self.adj_list[city1][city2] = distance
        self.adj_list[city2][city1] = distance

    def generate_random_graph(self): 
        for city1 in range(self.num_cities):
            for city2 in range(city1 + 1, self.num_cities):
                distance = random.randint(50, 100)
                self.add_distance(city1, city2, distance)

    def heuristic(self, current_city, visited): 
        min_distance = float('inf')
        nearest_neighbor = None

        for neighbor, distance in self.adj_list[current_city].items(): 
            if not visited[neighbor] and distance < min_distance: 
                min_distance = distance
                nearest_neighbor = neighbor
        return nearest_neighbor, min_distance

    def nearest_neighbor(self, start_city):
        visited = [False] * self.num_cities 
        path_nns = [start_city] 
        visited[start_city] = True
        visited_count = 1
        total_cost_nns = 0

        while visited_count < self.num_cities:
            current_city = path_nns[-1]
            nearest_neighbor, min_distance = self.heuristic(current_city, visited)
            path_nns.append(nearest_neighbor)
            visited[nearest_neighbor] = True
            visited_count += 1
            total_cost_nns += min_distance

        path_nns.append(start_city)
        total_cost_nns += self.adj_list[path_nns[-2]][start_city]
        return path_nns, total_cost_nns


In [32]:
num_cities = random.randint(4, 5)
NNS = NearestNeighbor(num_cities)
start_city = random.randint(0, NNS.num_cities - 1)
path_nns, total_cost_nns= NNS.nearest_neighbor(start_city)

In [36]:
print("Đường đi tìm được bằng NNS:")
print(" -> ".join(map(str, path_nns)))
print("Tổng quãng đường đi được (NNS):", total_cost_nns)

Đường đi tìm được bằng NNS:
3 -> 0 -> 1 -> 4 -> 2 -> 3
Tổng quãng đường đi được (NNS): 342


In [17]:
graph_data = {}
print("Đồ thị:")
for city, neighbors in NNS.adj_list.items():
    print(f"{city}: {neighbors}")
for city, neighbors in NNS.adj_list.items():
    new_neighbor = {}
    for neighbor, distance in neighbors.items():
        new_neighbor[neighbor] = distance
    graph_data[city] = new_neighbor


Đồ thị:
0: {1: 70, 2: 53, 3: 59, 4: 75}
1: {0: 70, 2: 98, 3: 73, 4: 70}
2: {0: 53, 1: 98, 3: 55, 4: 69}
3: {0: 59, 1: 73, 2: 55, 4: 65}
4: {0: 75, 1: 70, 2: 69, 3: 65}


GREEDY METHOD OR CLOSET PAIR

In [18]:
class Graph_CP:
    def __init__(self, vertices): 
        self.vertices = vertices

    def get_neighbors(self, v, edges):
        neighbors = []
        for edge in edges:
            if v not in edge:
                continue
            neighbors.append(edge[0] if edge[0] != v else edge[1])
        return neighbors

    def get_all_edges(self):
        edge_list = []
        for edge in itertools.combinations(self.vertices.keys(), 2):
            edge_list.append(edge)
        return edge_list

In [37]:
class Closet_Pair:
    def perform_algorithm(self, graph):
        all_edges = graph_cp.get_all_edges()
        edges_sorted = sorted(all_edges, key=lambda x: graph_cp.vertices[x[0]][x[1]])
        added_edges = []
        for edge in edges_sorted:
            degree_map = self.get_degree_map(graph, added_edges)
            if len(added_edges) == len(graph_cp.vertices) - 1:
                degrees_sorted = sorted(list(degree_map.keys()), key=lambda x: degree_map[x])
                final_edge = (degrees_sorted[0], degrees_sorted[1])
                added_edges.append(final_edge)
                break
            u, v = edge
            if degree_map[u] == 2 or degree_map[v] == 2:
                continue
            if self.is_connected(graph, u, v, added_edges):
                continue
            added_edges.append(edge)
            print("Added edge:", edge)
        print("Tất cả cạnh được thêm vào chu trình: ", added_edges)
        print("degree_map: ", degree_map)
        print("edges_sorted: ", edges_sorted)
        print("degrees_sorted: ", degrees_sorted)
        return added_edges

    def get_degree_map(self, graph, edges):
        v_to_degree = {v: 0 for v in graph_cp.vertices}
        for edge in edges:
            u, v = edge
            v_to_degree[u] = v_to_degree.get(u, 0) + 1 
            v_to_degree[v] = v_to_degree.get(v, 0) + 1
        return v_to_degree

    def is_connected(self, graph, u, v, edges):
        visited = set()
        def dfs(current):
            visited.add(current)
            for neighbor in graph_cp.get_neighbors(current, edges):
                if neighbor not in visited:
                    dfs(neighbor)
        dfs(u)
        return v in visited

    def calculate_total_cost(self, graph, edges):
        total_cost_gm = 0
        for edge in edges:
            u, v = edge
            total_cost_gm += graph_cp.vertices[u][v]
        return total_cost_gm


In [38]:
class HamiltonPath:
    def __init__(self, graph, start_city):
        self.graph = graph
        self.start_city = start_city

    def find_hamilton_path(self, added_edges):
        path = [self.start_city]
        visited = {self.start_city}
        self.dfs(self.start_city, path, visited, added_edges)
        path.append(self.start_city)
        return path

    def dfs(self, current_city, path, visited, added_edges):
        neighbors = self.graph.get_neighbors(current_city, added_edges) 
        for neighbor in neighbors:
            if neighbor not in visited:
                path.append(neighbor)
                visited.add(neighbor)
                self.dfs(neighbor, path, visited, added_edges)


In [39]:
graph_cp = Graph_CP(graph_data)
CP = Closet_Pair()
added_edges = CP.perform_algorithm(graph_cp)
total_cost_gm = CP.calculate_total_cost(graph_cp, added_edges)
print("Tổng quãng đường đã đi (GM): ", total_cost_gm)

Added edge: (0, 2)
Added edge: (2, 3)
Added edge: (3, 4)
Added edge: (0, 1)
Tất cả cạnh được thêm vào chu trình:  [(0, 2), (2, 3), (3, 4), (0, 1), (1, 4)]
degree_map:  {0: 2, 1: 1, 2: 2, 3: 2, 4: 1}
edges_sorted:  [(0, 2), (2, 3), (0, 3), (3, 4), (2, 4), (0, 1), (1, 4), (1, 3), (0, 4), (1, 2)]
degrees_sorted:  [1, 4, 0, 2, 3]
Tổng quãng đường đã đi (GM):  313


In [23]:
# Sau khi có chu trình Hamilton
path_find_gm = HamiltonPath(graph_cp, start_city)
path_gm = path_find_gm.find_hamilton_path(added_edges)
print("Đường đi tìm được bằng GM: ")
print(" -> ".join(map(str, path_gm)))

Đường đi tìm được bằng GM: 
2 -> 0 -> 1 -> 4 -> 3 -> 2


NEAREST ITERATION

In [24]:
def find_nearest_city(vertices, unvisited, path_nis):
        nearest_city = None
        min_distance = float('inf')

        for city in path_nis:
            for point in unvisited:
                distance = vertices[city][point]
                if distance < min_distance:
                    min_distance = distance
                    nearest_city = point
        return nearest_city

In [25]:
class NearestIteration:
    def __init__(self,vertices): 
        self.vertices = vertices

    def nis_heuristic(self, start_city):
        '''Tạo danh sách các điểm cần thăm'''
        unvisited = list(self.vertices.keys())
        unvisited.remove(start_city)
        path_nis = [start_city]

        while unvisited:
            nearest_city = find_nearest_city(self.vertices, unvisited, path_nis)
            unvisited.remove(nearest_city)

            if len(path_nis) >= 3:
                best_index = 1
                best_distance = self.vertices[path_nis[best_index - 1]][nearest_city]\
                            + self.vertices[nearest_city][path_nis[best_index]]\
                            - self.vertices[path_nis[best_index - 1]][path_nis[best_index]]
                for i in range(2, len(path_nis)): 
                    distance = self.vertices[path_nis[i - 1]][nearest_city]\
                            + self.vertices[nearest_city][path_nis[i]]\
                            - self.vertices[path_nis[i - 1]][path_nis[i]]
                    if distance < best_distance: 
                        best_index = i
                        best_distance = distance
                path_nis.insert(best_index, nearest_city)
            else:
                path_nis.append(nearest_city)

        path_nis.append(start_city)
        return path_nis
    
    def calculate_total_cost(self, path_nis):
        total_cost_nis = 0
        for i in range(len(path_nis) - 1):
            total_cost_nis += self.vertices[path_nis[i]][path_nis[i + 1]]
        return total_cost_nis


In [26]:
NIS = NearestIteration(graph_data)
path_nis = NIS.nis_heuristic(start_city)
total_cost_nis = NIS.calculate_total_cost(path_nis)

In [27]:
print("Đường đi tìm được bằng NIS:")
print(" -> ".join(map(str, path_nis)))

Đường đi tìm được bằng NIS:
2 -> 0 -> 1 -> 4 -> 3 -> 2


In [1786]:
print("Tổng quãng đường đi được (NIS): ", total_cost_nis)

Tổng quãng đường đi được (NIS):  733


BRUTE FORCE (Dùng để tìm tối ưu nhất. Từ đó suy ra hiệu suất)

In [28]:
class BF:
    def __init__(self, vertices):
        self.vertices = vertices

    def brute_force(self, start_city):
        num_cities = len(self.vertices)
        min_path = None
        min_cost = float('inf')
        cities = list(range(num_cities))
        cities.remove(start_city)  
        permutations = itertools.permutations(cities)

        for perm in permutations:
            path = [start_city] + list(perm) + [start_city]
            cost = self.get_cost(path)
            if cost < min_cost:
                min_cost = cost
                min_path = path
        return min_path, min_cost
    def get_cost(self, path):
        cost = 0
        for i in range(len(path) - 1):
            start_city = path[i]
            end_city = path[i + 1]
            cost += self.vertices[start_city][end_city]
        return cost


In [29]:
graph = BF(graph_data)
min_path, min_cost = graph.brute_force(start_city) 
print("Đường đi tối ưu:")
print(" -> ".join(map(str, min_path)))
print("Min cost:", min_cost)

Đường đi tối ưu:
2 -> 0 -> 1 -> 4 -> 3 -> 2
Min cost: 313


Tính hiệu suất

In [30]:
print(f"NNS: {total_cost_nns}, GM: {total_cost_gm}, NIS: {total_cost_nis}, Min cost: {min_cost} ")
def calculate_eff():
  x, y, t, z = total_cost_nns, total_cost_gm, total_cost_nis, min_cost
  return (x/z) * 100, (y/z) * 100, (t/z) * 100
eff_nns, eff_gm, eff_nis = calculate_eff()
print("Hiệu suất của NNS là: ",eff_nns)
print("Hiệu suất của GM là: ", eff_gm)
print("Hiệu suất của NIS là: ", eff_nis)

def compare_method():
    names = ['NNS', 'GM', 'NIS']
    costs = [total_cost_nns, total_cost_gm, total_cost_nis]
    max_index = min(range(len(costs)), key=lambda i: costs[i])
    highest_cost = costs[max_index]
    best_methods = [names[max_index]]
    for i in range(len(names)):
        if i != max_index:
            if costs[i] == highest_cost:
                best_methods.append(names[i])
            else:
                print(f"Chênh lệch quãng đường giữa {names[max_index]} và {names[i]} là {abs(highest_cost - costs[i])}")
    if len(best_methods) > 1:
        print(f"Các phương pháp có hiệu suất cao nhất là: {', '.join(best_methods)}")
    else:
        print(f"Phương pháp có hiệu suất cao nhất là {best_methods[0]}")
    return best_methods

best_eff = compare_method()

NNS: 345, GM: 313, NIS: 313, Min cost: 313 
Hiệu suất của NNS là:  110.22364217252397
Hiệu suất của GM là:  100.0
Hiệu suất của NIS là:  100.0
Chênh lệch quãng đường giữa GM và NNS là 32
Các phương pháp có hiệu suất cao nhất là: GM, NIS


In [1790]:
file_name = "result_greedy812.txt"
if not os.path.exists(file_name):
    count = 1
else:
    with open(file_name, "r") as f:
        first_line = f.readline().strip() 
        if first_line.startswith("-------- Result number"):
            count += 1

with open(file_name, "a") as f:
    f.write(f"-------- Result number {count} --------\n")
    f.write(f"Number of vertices: {num_cities}\n")
    f.write(f"NNS: {total_cost_nns}, GM: {total_cost_gm}, NIS: {total_cost_nis}, Min cost: {min_cost}\n")
    f.write(f"Eff of NNS: {eff_nns}\n")
    f.write(f"Eff of GM: {eff_gm}\n")
    f.write(f"Eff of NIS: {eff_nis}\n")
    f.write(f"Best eff: {best_eff}\n")

In [35]:
def count_best_eff(file_name):
    nns_count = 0
    gm_count = 0
    nis_count = 0

    with open(file_name, "r") as f:
        for line in f:
            if line.startswith("Best eff:"):
                if "NNS" in line:
                    nns_count += 1
                if "GM" in line:
                    gm_count += 1
                if "NIS" in line:
                    nis_count += 1

    return nns_count, gm_count, nis_count

total_nns_count = 0
total_gm_count = 0
total_nis_count = 0

file_names = ["result_greedy58.txt", "result_greedy812.txt"]
for file_name in file_names:
    nns_count, gm_count, nis_count = count_best_eff(file_name)
    total_nns_count += nns_count
    total_gm_count += gm_count
    total_nis_count += nis_count
    print(f"File: {file_name}")
    print(f"NNS count: {nns_count}")
    print(f"GM count: {gm_count}")
    print(f"NIS count: {nis_count}\n")
 
print(f"Total best eff NNS count: {total_nns_count}")
print(f"Total best eff GM count: {total_gm_count}")
print(f"Total best eff NIS count: {total_nis_count}")


File: result_greedy58.txt
NNS count: 19
GM count: 25
NIS count: 13

File: result_greedy812.txt
NNS count: 10
GM count: 11
NIS count: 4

Total best eff NNS count: 29
Total best eff GM count: 36
Total best eff NIS count: 17
